In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib widget
plt.ion()
sns.set(rc={'figure.figsize': (11, 4)})
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [5]:
# Load data
df = pd.read_csv("../data/imputed/data_imp1.csv", index_col=0)
df = df.dropna()

In [ ]:
metadata = pd.read_csv("../data/")

In [6]:
# Group data by sitename
sites = df['sitename'].unique()
sites_df = [df[df['sitename'] == site] for site in sites]

for i in range(len(sites_df)):
    sites_df[i]['date'] = pd.to_datetime(sites_df[i]['date'], format="%Y-%m-%d")
    sites_df[i] = sites_df[i].set_index("date")

In [314]:
df

,sitename,date,TA_F,SW_IN_F,LW_IN_F,VPD_F,PA_F,P_F,WS_F,USTAR,CO2_F_MDS,GPP_NT_VUT_REF
1,AR-Vir,2009-01-01,28.671,307.007,392.519,20.158,99.548,0.000,3.744,0.553115,354.284918,-0.695432
2,AR-Vir,2009-01-02,24.547,212.807,399.500,11.046,99.552,6.917,3.913,0.534813,349.998459,9.582348
3,AR-Vir,2009-01-03,22.329,276.267,369.261,11.534,99.733,0.329,4.206,0.624032,385.025344,8.512268
4,AR-Vir,2009-01-04,22.994,339.802,335.382,14.660,99.979,0.000,2.745,0.136002,387.550232,5.998422
5,AR-Vir,2009-01-05,24.699,345.667,339.134,17.114,99.785,0.000,1.799,0.241101,361.244943,9.369482
...,...,...,...,...,...,...,...,...,...,...,...,...
265173,ZM-Mon,2009-12-27,22.857,291.147,375.190,7.918,89.212,0.467,1.897,0.495470,393.599670,5.688883
265174,ZM-Mon,2009-12-28,22.720,271.966,381.716,6.998,89.171,1.650,1.360,0.396692,370.456133,7.266249
265175,ZM-Mon,2009-12-29,21.899,241.666,381.685,4.748,89.142,5.493,1.403,0.207637,445.700182,9.269704
265176,ZM-Mon,2009-12-30,22.296,234.093,384.903,6.234,89.143,2.134,2.160,0.256847,363.831560,12.287981


In [320]:
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from tqdm import tqdm

regr =  GradientBoostingRegressor(n_estimators=100, learning_rate=0.1,  max_depth=2, random_state=0, loss='ls')
cv = []
sites = df['sitename'].unique()


for i in tqdm(range(len(sites))):
    train = df[df.sitename != sites[i]]
    test = df[df.sitename == sites[i]]
    X_train = train.drop(columns=["GPP_NT_VUT_REF","sitename","date"]).values
    X_test = test.drop(columns=["GPP_NT_VUT_REF","sitename","date"]).values
    y_train = train['GPP_NT_VUT_REF'].values
    y_test = test['GPP_NT_VUT_REF'].values
    regr.fit(X_train,y_train)
    preds = regr.predict(X_test)
    cv.append((sites[i],mean_squared_error(y_test,preds)))




100%|██████████| 71/71 [43:26<00:00, 36.72s/it]


In [347]:
cv_np = np.asarray(cv)[:,1].astype(float)
print(f"LOOCV: {np.mean(cv_np)} +- {np.std(cv_np)}")

LOOCV: 7.514659705557011 +- 3.410238044429465


In [348]:
cv

[('AR-Vir', 21.449606436009276),
 ('AU-Ade', 6.585498572669307),
 ('AU-ASM', 8.478658697482906),
 ('AU-DaP', 8.689628473455361),
 ('AU-DaS', 4.276220110219993),
 ('AU-Dry', 5.683514302578594),
 ('AU-Fog', 12.964600786247477),
 ('AU-Gin', 9.156615376546563),
 ('AU-How', 5.097902475703656),
 ('AU-Stp', 7.978873400740239),
 ('AU-Whr', 5.668417771188435),
 ('AU-Wom', 5.251263219856007),
 ('BE-Bra', 4.092026824358347),
 ('BE-Vie', 5.194755408038089),
 ('CH-Fru', 10.987115726972805),
 ('CH-Lae', 8.537411966640988),
 ('CH-Oe1', 8.585237533954537),
 ('CN-Cng', 6.047398278622873),
 ('CN-Qia', 4.8505266910465),
 ('CZ-wet', 4.828390255929098),
 ('DE-Akm', 7.7415583865818824),
 ('DE-Geb', 12.240551495559728),
 ('DE-Gri', 6.771816027849155),
 ('DE-Hai', 8.47595586505541),
 ('DE-Kli', 10.847627383791696),
 ('DE-Obe', 7.486132184440825),
 ('DE-RuR', 5.512538378890511),
 ('DE-Spw', 8.242915615079932),
 ('DE-Tha', 7.705782297063466),
 ('DK-NuF', 9.514240187593197),
 ('DK-Sor', 14.301144898250872),
 ('F

In [306]:
X_test = sites_df[53].drop(columns=["GPP_NT_VUT_REF","sitename"]).values
y_test = sites_df[53]['GPP_NT_VUT_REF']
preds = regr.predict(X_test)

In [307]:
%matplotlib widget
index = sites_df[53].index
plt.ion()
plt.scatter(index, preds, label="Predictions", s=1)
plt.scatter(index, y_test, label="Ground truth"+" "+sites_df[65]['sitename'].unique(), s=1)
plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [304]:
residuals = (preds - y_test) ** 2

%matplotlib widget
plt.ion()
plt.scatter(index, residuals, label="Residuals", s=1)
plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [305]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test,preds)

10.983337980311918